# Генерация анекдотов с дообученной моделью (локальный запуск)

Ноутбук для экспериментов с уже дообученной моделью на локальном ПК.

In [ ]:
import torch
import re
import random

print(f"PyTorch: {torch.__version__}")

if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print(f"Device: {device}")

PyTorch: 2.9.1
Device: mps


## 1. Загрузка модели

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch
import re
import os

os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

BASE_MODEL_PATH = "./qwen3_model"
LORA_PATH = "./qwen3_jokes_lora_2"

print("Загрузка базовой модели...")
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_PATH,
    torch_dtype=torch.float32,
    trust_remote_code=True,
    local_files_only=True,
    low_cpu_mem_usage=True,
).to("cpu")

print("Загрузка токенайзера...")
tokenizer = AutoTokenizer.from_pretrained(
    BASE_MODEL_PATH,
    trust_remote_code=True,
    local_files_only=True,
)
tokenizer.pad_token = tokenizer.eos_token

print("Загрузка LoRA...")
model = PeftModel.from_pretrained(base_model, LORA_PATH).to("cpu")
model.eval()

print("Модель загружена на CPU")

Загрузка базовой модели...
Загрузка токенайзера...
Загрузка LoRA...
Модель загружена на CPU


## 2. Функция генерации

In [ ]:
import random
import re
import os

prefixes = []
with open("prefixes.txt", "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if line:
            parts = line.split(' ', 1)
            if len(parts) > 1 and parts[0].isdigit():
                prefixes.append(parts[1])
            else:
                prefixes.append(line)

print(f"Загружено префиксов: {len(prefixes)}")

def generate_for_prefix(prefix, max_tokens=60):
    messages = [
        {"role": "user", "content": f"Закончи анекдот: {prefix} /no_think"}
    ]
    
    input_text = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    inputs = tokenizer(input_text, return_tensors="pt")
    
    torch.manual_seed(random.randint(0, 1000000))
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            do_sample=True,
            temperature=random.uniform(0.7, 0.9),
            top_p=0.9,
            repetition_penalty=1.3,
            pad_token_id=tokenizer.eos_token_id,
        )
    
    new_tokens = outputs[0][inputs['input_ids'].shape[1]:]
    ending = tokenizer.decode(new_tokens, skip_special_tokens=True).strip()
    
    for end_char in ['.', '!', '?']:
        idx = ending.find(end_char)
        if 0 < idx < 100:
            ending = ending[:idx+1]
            break
    
    ending = re.sub(r'<think>.*?</think>', '', ending, flags=re.DOTALL).strip()
    ending = ending.split('\n')[0].strip()
    
    return f"{prefix} {ending}"

def safe_filename(prefix):
    name = prefix[:40].strip()
    name = re.sub(r'[^\w\s-]', '', name)
    name = re.sub(r'\s+', '_', name)
    return name

os.makedirs("jokes_by_prefix", exist_ok=True)

VARIANTS_PER_PREFIX = 100

print(f"   Генерация {VARIANTS_PER_PREFIX} вариантов для {len(prefixes)} префиксов")
print(f"   Всего: ~{VARIANTS_PER_PREFIX * len(prefixes)} анекдотов\n")

for i, prefix in enumerate(prefixes):
    filename = f"jokes_by_prefix/{i+1:02d}_{safe_filename(prefix)}.txt"
    
    jokes = set()
    attempts = 0
    
    while len(jokes) < VARIANTS_PER_PREFIX and attempts < VARIANTS_PER_PREFIX * 3:
        joke = generate_for_prefix(prefix)
        if len(joke) > len(prefix) + 10:
            jokes.add(joke)
        attempts += 1
    
    with open(filename, "w", encoding="utf-8") as f:
        f.write(f"# Префикс: {prefix}\n")
        f.write(f"# Вариантов: {len(jokes)}\n\n")
        for j, joke in enumerate(jokes, 1):
            f.write(f"{j}. {joke}\n\n")
    
    print(f"[{i+1}/{len(prefixes)}] {prefix[:35]}... → {len(jokes)}")

print(f"\nГотово! Файлы в jokes_by_prefix/")

Загружено префиксов: 75
   Генерация 100 вариантов для 75 префиксов
   Всего: ~7500 анекдотов

[1/75] Идёт мужик по лесу... → 100
[2/75] Встречаются два друга... → 100
[3/75] Приходят мужик в бар... → 100
[4/75] Жена говорит мужу... → 100
[5/75] Приходят альфа, бета и гамма в бар... → 100
[6/75] Идёт медведь по лесу... → 100
[7/75] Приходит мужик к врачу... → 100
[8/75] Встречаются русский, американец и н... → 100
[9/75] Идёт по улице девушка... → 100
[10/75] Приходит мужик в магазин... → 100
[11/75] Еще сто лет назад... → 100
[12/75] Встречаются Вовочка и Петька... → 100
[13/75] Идёт по лесу охотник... → 100
[14/75] Я хорошо готовлю, стираю и убираю в... → 100
[15/75] Жена спрашивает у мужа... → 100
[16/75] Сидят в баре два друга... → 100
[17/75] Идёт по пустыне караван... → 100
[18/75] Приходит мужик в аптеку... → 100
[19/75] Встречаются два программиста... → 100
[20/75] - Послушайте, у этого парня в резюм... → 100
[21/75] Приходит мужик в банк... → 100
[22/75] Сидят на скамейке два 

## Справка по параметрам

| Параметр | Диапазон | Эффект |
|----------|----------|--------|
| `temperature` | 0.5-1.2 | Ниже = предсказуемо, выше = креативно |
| `top_p` | 0.7-0.95 | Nucleus sampling, ниже = консервативнее |
| `top_k` | 20-100 | Ограничивает выбор токенов |
| `repetition_penalty` | 1.0-1.5 | Выше = меньше повторов |
| `max_tokens` | 100-300 | Максимальная длина |
